Implementing D-Tree in python using numpy with entropy and Information Gain Values

In [1]:
'''
DATASET : 

age	income	student	credit	buys_computer
<=30	high	no 	fair	no
<=30	high	no 	good	no
31..40	high	no 	fair	yes
>40	medium	no 	fair	yes
>40	low	yes	fair	yes
>40	low	yes	good	no
31..40	low	yes	good	yes
<=30	medium	no 	fair	no
<=30	low	yes	fair	yes
>40	medium	yes	fair	yes
<=30	medium	yes	good	yes
31..40	medium	no 	good	yes
31..40	high	yes	fair	yes
>40	medium	no 	good	no

'''

'\nDATASET : \n\nage\tincome\tstudent\tcredit\tbuys_computer\n<=30\thigh\tno \tfair\tno\n<=30\thigh\tno \tgood\tno\n31..40\thigh\tno \tfair\tyes\n>40\tmedium\tno \tfair\tyes\n>40\tlow\tyes\tfair\tyes\n>40\tlow\tyes\tgood\tno\n31..40\tlow\tyes\tgood\tyes\n<=30\tmedium\tno \tfair\tno\n<=30\tlow\tyes\tfair\tyes\n>40\tmedium\tyes\tfair\tyes\n<=30\tmedium\tyes\tgood\tyes\n31..40\tmedium\tno \tgood\tyes\n31..40\thigh\tyes\tfair\tyes\n>40\tmedium\tno \tgood\tno\n\n'

In [2]:
import numpy as np
x1 = [1,1,2,3,3,3,2,1,1,3,1,2,2,3] #age attribute 1 : age<=30, 2 :31-40 ,3: >40.
x2 = [3,3,3,2,1,1,1,2,1,2,2,2,3,2] #income 1: low, 2 : medium ,3 :high.
x3=  [2,2,2,2,1,1,1,2,1,1,1,2,1,2] #student 1:yes, 2:no
x4 = [1,2,1,1,1,2,2,1,1,1,2,2,1,2] #credit 1:fair, 2:good


y = np.array([2,2,1,1,1,2,1,2,1,1,1,1,1,2]) #buys_computer 1:yes 2:no TARGET LABEL


In [3]:
#splits the target label
def partition(a):
    return {c: (a==c).nonzero()[0] for c in np.unique(a)}

In [4]:
#calculates class entropy I(9,5)
def entropy(s):
    res = 0
    val, counts = np.unique(s, return_counts=True)
    freqs = counts.astype('float')/len(s)
    for p in freqs:
        if p != 0.0:
            res -= p * np.log2(p)
    return res

In [5]:
#claculates attribute entropy (example Iage(D)=5/14*I(2,3)+4/14*I(4,0))
def mutual_information(y, x):

    res = entropy(y)

    # We partition x, according to attribute values x_i
    val, counts = np.unique(x, return_counts=True)
    freqs = counts.astype('float')/len(x)

    # We calculate a weighted average of the entropy
    for p, v in zip(freqs, val):
        res -= p * entropy(y[x == v])

    return res

In [6]:
from pprint import pprint

#if we cannot categorize further.
def is_pure(s):
    return len(set(s)) == 1

def recursive_split(x, y):
    # If there could be no split, just return the original set
    if is_pure(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    gain = np.array([mutual_information(y, x_attr) for x_attr in x.T])
    selected_attr = np.argmax(gain)

    # If there's no gain at all, nothing has to be done, just return the original set
    if np.all(gain < 1e-6):
        return y


    # We split using the selected attribute
    sets = partition(x[:, selected_attr])

    res = {}
    for k, v in sets.items():
        y_subset = y.take(v, axis=0)
        x_subset = x.take(v, axis=0)

        res["x_%d = %d" % (selected_attr, k)] = recursive_split(x_subset, y_subset)

    return res

X = np.array([x1, x2, x3, x4]).T
print("x_0 : AGE, x_1 : INCOME, x_2 STUDENT, x_3 : CREDIT \n")
print ("array([1,2]) represents class label 1-yes 2-no \n")
pprint(recursive_split(X, y))

x_0 : AGE, x_1 : INCOME, x_2 STUDENT, x_3 : CREDIT 

array([1,2]) represents class label 1-yes 2-no 

{'x_0 = 1': {'x_2 = 1': array([1, 1]), 'x_2 = 2': array([2, 2, 2])},
 'x_0 = 2': array([1, 1, 1, 1]),
 'x_0 = 3': {'x_3 = 1': array([1, 1, 1]), 'x_3 = 2': array([2, 2])}}
